In [256]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from numpy import genfromtxt

In [257]:
d = pd.read_csv('letter-recognition.data', sep=",")
d.head()

,lettr,x-box,y-box,width,high,onpix,x-bar,y-bar,x2bar,y2bar,xybar,x2ybr,xy2br,x-ege,xegvy,y-ege,yegvx
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10


In [353]:
def loadData(filename):  
   data = genfromtxt(filename, delimiter=',',dtype=None,encoding="UTF8")[1:]
   X = data[:,1:].astype('int32')
   y = data[:,0]
   return X, y

X, y = loadData('letter-recognition.data')

In [259]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)
y_transformed = le.transform(y)

In [260]:
from sklearn.model_selection import train_test_split
# u članku podjela 15000:5000, a kasnije piše 60%:40%
X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size=0.25, random_state=42)

In [330]:
clf = RandomForestClassifier(n_estimators=100, max_features="sqrt", max_depth=25, min_samples_split=5)
clf.fit(X_train, y_train)
decision_trees = clf.estimators_

In [266]:
def mapNodeIndicesToLeafIndices(tree):
   mapping = dict()
   node_index, leafIndex = 0, 0
   def createMapping(tree, node_index, leafIndex):
      if tree.tree_.children_left[node_index] == -1:
         mapping[node_index] = leafIndex
         leafIndex = leafIndex + 1
         return leafIndex
      leafIndex = createMapping(tree, tree.tree_.children_left[node_index], leafIndex)
      leafIndex = createMapping(tree, tree.tree_.children_right[node_index], leafIndex)
      return leafIndex
   
   createMapping(tree, node_index, leafIndex)
   return mapping

def getListOfMaps(forest):
   return [mapNodeIndicesToLeafIndices(tree) for tree in forest]

In [349]:
listOfIndicesMaps = getListOfMaps(decision_trees)

In [268]:
def getAllLeavesNumeration(listOfIndicesMaps):
   num_of_all_forest_leaves = 0
   num_of_leaves_so_far = []
   n_leaves = len(listOfIndicesMaps[0])
   num_of_leaves_so_far.append(n_leaves)
   num_of_all_forest_leaves += n_leaves
   for t in range(1, len(listOfIndicesMaps), 1):
      n_leaves = len(listOfIndicesMaps[t])
      num_of_leaves_so_far.append(num_of_leaves_so_far[-1] + n_leaves)
      num_of_all_forest_leaves += n_leaves
   return num_of_leaves_so_far, num_of_all_forest_leaves

In [269]:
from scipy.sparse import csr_matrix

#creates a SPARE matrix of dimension: num_of_instances x num_of_all_forest_leaves
def create_Phi_matrix(X, decision_trees, listOfIndicesMaps):
   #dim (br.stabala x br.primjera)
   leaf_id_T = np.array([d.apply(X) for d in decision_trees])
   #unutra su node_indices, želimo leaf_indices
   for t in range(len(leaf_id_T)):
      tree_map = listOfIndicesMaps[t]
      for i in range(len(X)):
         leaf_id_T[t][i] = tree_map[leaf_id_T[t][i]]
   num_of_leaves_so_far, num_of_all_forest_leaves = getAllLeavesNumeration(listOfIndicesMaps)
   row, col = np.array([]), np.array([])
   for i in range(len(decision_trees)):
      row = np.concatenate((row, np.arange(len(X))))
   col = np.concatenate((col, leaf_id_T[0]))
   for t in range(1, len(leaf_id_T), 1):
      col = np.concatenate((col, leaf_id_T[t] + num_of_leaves_so_far[t-1]))
   data = np.array([1 for i in range(len(X)*len(decision_trees))])
   #print(len(row), len(col), len(data))
   Phi_matrix = csr_matrix((data, (np.array(row), np.array(col))), shape=(len(X), num_of_all_forest_leaves))
   return Phi_matrix

In [350]:
Phi_train = create_Phi_matrix(X_train, decision_trees, listOfIndicesMaps)
Phi_test = create_Phi_matrix(X_test, decision_trees, listOfIndicesMaps)

In [392]:
from liblinear.liblinearutil import *

def optimize(y_train, Phi_train, classification=False, regression=False):
   prob  = problem(y_train, Phi_train)
   if classification:
      param = parameter('-s 2 -c 0.1')
   if regression:
      param = parameter('-s 11 -c 0.1')
   model = train(prob, param)
   [W, b] = model.get_decfun()
   return model, W

In [393]:
model, W = optimize(y_train, Phi_train, classification=True)

In [352]:
y_test_predicted,_,_ = predict(y_test, Phi_test, model)

Accuracy = 96.46% (4823/5000) (classification)


In [361]:
import math

def findAdjacentLeaves(decision_trees, listOfIndicesMaps, W):
   #print(len(W))
   adjacent_leaves = []
   n = 0
   #num_of_leaves_so_far, _ =  getAllLeavesNumeration(listOfIndicesMaps)
   #num_of_leaves_so_far = [0] + num_of_leaves_so_far
 #....
   for t in range(0,len(decision_trees)):
      tree_map = listOfIndicesMaps[t]
      children_right = decision_trees[t].tree_.children_right
      for node_index in tree_map:
         sibling_node_index = children_right[node_index-1]
         if(sibling_node_index in tree_map):
            #sibling is a leaf
            #for each pair of adjacent leaves, we first compute the summation of the l2-norm of their leaf vectors to measure the significance
            #print(num_of_leaves_so_far[t-1] + tree_map[node_index])
            leaf_vector1 = W[n + tree_map[node_index]]
            leaf_vector2 = W[n + tree_map[sibling_node_index]]
            significance = math.sqrt(math.pow(leaf_vector1, 2)) + math.sqrt(math.pow(leaf_vector2, 2))
            adjacent_leaves.append([t, node_index, sibling_node_index, significance])
      n+=len(listOfIndicesMaps[t])
   return adjacent_leaves


In [346]:
adjacent_leaves = findAdjacentLeaves(decision_trees, listOfIndicesMaps, W)
#sortiraj parove listova po signifikantnosti
adjacent_leaves = sorted(adjacent_leaves, key=lambda x: x[3])
#print(adjacent_leaves)

129135


In [347]:
#we empirically prune 10% of leaves in each iteration
num_of_pairs_to_prune = round(0.1 * len(W))
pairs_to_prune = adjacent_leaves[:num_of_pairs_to_prune]
print(pairs_to_prune[0])

[89, 1850, 1851, 0.0006692291626278761]


In [325]:
#pruning:  two adjacent leaves are merged into one new leaf if the norm of their leaf vectors are close to zeros
def pruneTrees(pairs_to_prune, decision_trees):
   for p in pairs_to_prune:
      t = p[0]
      parent_index = p[1] - 1
      decision_trees[t].tree_.children_left[parent_index] = -1
      decision_trees[t].tree_.children_right[parent_index] = -1
   return decision_trees

In [348]:
decision_trees = pruneTrees(pairs_to_prune, decision_trees)

In [416]:
def calculate_refined_A(X, y, maxIterations, classification=False, regression=False, splitData = True):
   if splitData:
      le.fit(y)
      y_transformed = le.transform(y)
      X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size=0.25, random_state=42)
   else:
      # X , y
      #[X_train, X_test], [y_train, y_test]
      X_train = X[0]
      X_test = X[1]
      y_train = y[0]
      y_test = y[1]
   clf = RandomForestClassifier(n_estimators=100, max_features="sqrt", max_depth=25, min_samples_split=5)
   clf.fit(X_train, y_train)
   decision_trees = clf.estimators_
   i = 0
   while i < maxIterations:
      i += 1
      print("iteration " + str(i) + ":    ", end="")
      listOfIndicesMaps = getListOfMaps(decision_trees)
      Phi_train = create_Phi_matrix(X_train, decision_trees, listOfIndicesMaps)
      Phi_test = create_Phi_matrix(X_test, decision_trees, listOfIndicesMaps)
      model, W = optimize(y_train, Phi_train, classification, regression)
      predict(y_test, Phi_test, model)
      adjacent_leaves = findAdjacentLeaves(decision_trees, listOfIndicesMaps, W)
      adjacent_leaves = sorted(adjacent_leaves, key=lambda x: x[3])
      num_of_pairs_to_prune = round(0.1 * len(W))
      pairs_to_prune = adjacent_leaves[:num_of_pairs_to_prune]
      decision_trees = pruneTrees(pairs_to_prune, decision_trees)
      

In [408]:
X, y = loadData('letter-recognition.data')
calculate_refined_A(X, y, 5, classification=True)

iteration 1:    Accuracy = 96.58% (4829/5000) (classification)
iteration 2:    Accuracy = 96.58% (4829/5000) (classification)
iteration 3:    Accuracy = 96.62% (4831/5000) (classification)
iteration 4:    Accuracy = 96.5% (4825/5000) (classification)
iteration 5:    Accuracy = 96.56% (4828/5000) (classification)


In [417]:
image_size = 28
train_data_count = 60_000
test_data_count = 10_000

import gzip

def readGzImages(path,number_of_images):
    input_X = gzip.open(path,'r')
    input_X.read(16)
    buff_X = input_X.read(image_size * image_size * number_of_images)
    X = np.frombuffer(buff_X, dtype=np.uint8).astype(np.float32)
    X = X.reshape(number_of_images ,image_size * image_size)
    return X

def readGzLabels(path, number_of_labels):
    input_y = gzip.open(path,'r')
    input_y.read(8)
    buff_y = input_y.read(1 * number_of_labels)
    y = np.frombuffer(buff_y, dtype=np.uint8).astype(np.int32)
    y = y.reshape(number_of_labels)
    return y

X_train = readGzImages('train-images-idx3-ubyte.gz', train_data_count)
y_train = readGzLabels('train-labels-idx1-ubyte.gz', train_data_count)
X_test = readGzImages('t10k-images-idx3-ubyte.gz', test_data_count)
y_test = readGzLabels('t10k-labels-idx1-ubyte.gz', test_data_count)
calculate_refined_A([X_train, X_test], [y_train, y_test], 5, classification=True, splitData=False)


iteration 1:    Accuracy = 97.73% (9773/10000) (classification)
iteration 2:    Accuracy = 97.75% (9775/10000) (classification)
iteration 3:    Accuracy = 97.72% (9772/10000) (classification)
iteration 4:    Accuracy = 97.74% (9774/10000) (classification)
iteration 5:    

In [398]:
data = genfromtxt('abalone.data', delimiter=',',dtype=str,encoding="UTF8")
X = data[:,1:].astype('float32')
y = data[:,0]
calculate_refined_A(X, y, 10, regression=True)

iteration 1:    Mean squared error = 0.756639 (regression)
Squared correlation coefficient = 0.000666831 (regression)
iteration 2:    Mean squared error = 0.757375 (regression)
Squared correlation coefficient = 0.000701887 (regression)
iteration 3:    Mean squared error = 0.761255 (regression)
Squared correlation coefficient = 0.000666018 (regression)
iteration 4:    Mean squared error = 0.767516 (regression)
Squared correlation coefficient = 0.000357401 (regression)
iteration 5:    Mean squared error = 0.770906 (regression)
Squared correlation coefficient = 0.000423992 (regression)
iteration 6:    Mean squared error = 0.777268 (regression)
Squared correlation coefficient = 0.000370787 (regression)
iteration 7:    Mean squared error = 0.777716 (regression)
Squared correlation coefficient = 0.000778296 (regression)
iteration 8:    Mean squared error = 0.779471 (regression)
Squared correlation coefficient = 0.00108157 (regression)
iteration 9:    Mean squared error = 0.790662 (regression

In [400]:
data = genfromtxt('ailerons.data', delimiter=',',dtype=str,encoding="UTF8")
X = data[:,:-1].astype('float32')
y = data[:,-1]
calculate_refined_A(X, y, 10, regression=True)

iteration 1:    Mean squared error = 3.39503 (regression)
Squared correlation coefficient = 0.807951 (regression)
iteration 2:    Mean squared error = 3.38168 (regression)
Squared correlation coefficient = 0.808616 (regression)
iteration 3:    Mean squared error = 3.3702 (regression)
Squared correlation coefficient = 0.809032 (regression)
iteration 4:    Mean squared error = 3.37434 (regression)
Squared correlation coefficient = 0.808481 (regression)
iteration 5:    Mean squared error = 3.38505 (regression)
Squared correlation coefficient = 0.807576 (regression)
iteration 6:    Mean squared error = 3.37939 (regression)
Squared correlation coefficient = 0.807327 (regression)
iteration 7:    Mean squared error = 3.39849 (regression)
Squared correlation coefficient = 0.805697 (regression)
iteration 8:    Mean squared error = 3.36293 (regression)
Squared correlation coefficient = 0.807608 (regression)
iteration 9:    Mean squared error = 3.37565 (regression)
Squared correlation coefficient